In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("News").getOrCreate()
sc    = spark.sparkContext      # get the context
spark
sc

<SparkContext master=yarn appName=News>

In [2]:
# Example uses GDELT dataset found here: https://aws.amazon.com/public-datasets/gdelt/
# Column headers found here: http://gdeltproject.org/data/lookups/CSV.header.dailyupdates.txt

# Load RDD
lines = sc.textFile("s3://gdelt-open-data/events/2016*") # Loads 73,385,698 records from 2016

In [3]:
# Split lines into columns; change split() argument depending on deliminiter e.g. '\t'
parts = lines.map(lambda l: l.split('\t'))

In [4]:
# Convert RDD into DataFrame
from urllib.request import urlopen

In [5]:
html = sc.textFile("http://gdeltproject.org/data/lookups/CSV.header.dailyupdates.txt")

In [6]:
columns = html.map(lambda l: l.split('\t'))

In [7]:
df = parts.toDF(schema=["GLOBALEVENTID", "SQLDATE", 'MonthYear','Year','FractionDate','Actor1Code','Actor1Name','Actor1CountryCode',
                'Actor1KnownGroupCode','Actor1EthnicCode',
               'Actor1Religion1Code','Actor1Religion2Code','Actor1Type1Code','Actor1Type2Code','Actor1Type3Code','Actor2Code',
                'Actor2Name','Actor2CountryCode',
               'Actor2KnownGroupCode','Actor2EthnicCode','Actor2Religion1Code','Actor2Religion2Code','Actor2Type1Code',
                'Actor2Type2Code','Actor2Type3Code','IsRootEvent','EventCode',
               'EventBaseCode','EventRootCode','QuadClass','GoldsteinScale','NumMentions','NumSources','NumArticles',
                'AvgTone','Actor1Geo_Type','Actor1Geo_FullName','Actor1Geo_CountryCode',
               'Actor1Geo_ADM1Code','Actor1Geo_Lat','Actor1Geo_Long','Actor1Geo_FeatureID','Actor2Geo_Type','Actor2Geo_FullName',
                'Actor2Geo_CountryCode','Actor2Geo_ADM1Code','Actor2Geo_Lat',
               'Actor2Geo_Long','Actor2Geo_FeatureID','ActionGeo_Type','ActionGeo_FullName','ActionGeo_CountryCode',
                'ActionGeo_ADM1Code','ActionGeo_Lat','ActionGeo_Long','ActionGeo_FeatureID','DATEADDED','SOURCEURL'])

In [8]:
df.take(1)

[Row(GLOBALEVENTID='498554164', SQLDATE='20060103', MonthYear='200601', Year='2006', FractionDate='2006.0082', Actor1Code='BUS', Actor1Name='CORPORATION', Actor1CountryCode='', Actor1KnownGroupCode='', Actor1EthnicCode='', Actor1Religion1Code='', Actor1Religion2Code='', Actor1Type1Code='BUS', Actor1Type2Code='', Actor1Type3Code='', Actor2Code='', Actor2Name='', Actor2CountryCode='', Actor2KnownGroupCode='', Actor2EthnicCode='', Actor2Religion1Code='', Actor2Religion2Code='', Actor2Type1Code='', Actor2Type2Code='', Actor2Type3Code='', IsRootEvent='1', EventCode='051', EventBaseCode='051', EventRootCode='05', QuadClass='1', GoldsteinScale='3.4', NumMentions='6', NumSources='2', NumArticles='6', AvgTone='1.72415843232381', Actor1Geo_Type='4', Actor1Geo_FullName='American Creek, British Columbia, Canada', Actor1Geo_CountryCode='CA', Actor1Geo_ADM1Code='CA02', Actor1Geo_Lat='49.05', Actor1Geo_Long='-116', Actor1Geo_FeatureID='-560119', Actor2Geo_Type='0', Actor2Geo_FullName='', Actor2Geo_Co

In [76]:
#### no need to run for re-running my code
df.count()

73385698

In [78]:
#### no need to run for re-running my code
df.printSchema()

root
 |-- GLOBALEVENTID: string (nullable = true)
 |-- SQLDATE: string (nullable = true)
 |-- MonthYear: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- FractionDate: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1Type1Code: string (nullable = true)
 |-- Actor1Type2Code: string (nullable = true)
 |-- Actor1Type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2Name: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCode: string (nullable = true)
 |-- Actor2EthnicCode: string (nullable = true)
 |-- Actor2Religion1Code: string (nullable = true)
 |-- Actor2Religion2Code: stri

In [9]:
# register a dataframe before running the sql
df.createOrReplaceTempView("df")

In [84]:
#### no need to run for re-running my code
df.take(5)

[Row(GLOBALEVENTID='498554164', SQLDATE='20060103', MonthYear='200601', Year='2006', FractionDate='2006.0082', Actor1Code='BUS', Actor1Name='CORPORATION', Actor1CountryCode='', Actor1KnownGroupCode='', Actor1EthnicCode='', Actor1Religion1Code='', Actor1Religion2Code='', Actor1Type1Code='BUS', Actor1Type2Code='', Actor1Type3Code='', Actor2Code='', Actor2Name='', Actor2CountryCode='', Actor2KnownGroupCode='', Actor2EthnicCode='', Actor2Religion1Code='', Actor2Religion2Code='', Actor2Type1Code='', Actor2Type2Code='', Actor2Type3Code='', IsRootEvent='1', EventCode='051', EventBaseCode='051', EventRootCode='05', QuadClass='1', GoldsteinScale='3.4', NumMentions='6', NumSources='2', NumArticles='6', AvgTone='1.72415843232381', Actor1Geo_Type='4', Actor1Geo_FullName='American Creek, British Columbia, Canada', Actor1Geo_CountryCode='CA', Actor1Geo_ADM1Code='CA02', Actor1Geo_Lat='49.05', Actor1Geo_Long='-116', Actor1Geo_FeatureID='-560119', Actor2Geo_Type='0', Actor2Geo_FullName='', Actor2Geo_Co

## Data Cleansing

In [10]:
# GoldsteinScale, AvgTone, Actor1Geo_Lat, Actor1Geo_Long should be decimals
df1 = df.withColumn("GoldsteinScale",df["GoldsteinScale"].cast('float'))
df1 = df1.withColumn("AvgTone",df["AvgTone"].cast('float'))  
df1 = df1.withColumn("Actor1Geo_Lat",df["Actor1Geo_Lat"].cast('float'))  
df1 = df1.withColumn("Actor1Geo_Long",df["Actor1Geo_Long"].cast('float'))

In [11]:
from pyspark.sql.types import IntegerType

# NumMentions, NumSources, NumArticles should be integer
df1 = df1.withColumn("NumMentions", df["NumMentions"].cast(IntegerType()))
df1 = df1.withColumn("NumSources", df["NumSources"].cast(IntegerType()))
df1 = df1.withColumn("NumArticles", df["NumArticles"].cast(IntegerType()))

In [16]:
# convert SQLDATE to DateType() if they are StringType()
from pyspark.sql.functions import to_date
df1 = df1.withColumn('SQLDATE', to_date('SQLDATE'))

In [17]:
# no need to remove missing values for analysis because the event happend anyways and maybe there is only 1 actor
# by may consider remove missing values for models building. To remove:
# df.na.drop()

In [18]:
#### no need to run for re-running my code
df1.printSchema()

root
 |-- GLOBALEVENTID: string (nullable = true)
 |-- SQLDATE: date (nullable = true)
 |-- MonthYear: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- FractionDate: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1Type1Code: string (nullable = true)
 |-- Actor1Type2Code: string (nullable = true)
 |-- Actor1Type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2Name: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCode: string (nullable = true)
 |-- Actor2EthnicCode: string (nullable = true)
 |-- Actor2Religion1Code: string (nullable = true)
 |-- Actor2Religion2Code: string

In [90]:
#### no need to run for re-running my code
# check for missing values in each column
from pyspark.sql.functions import isnan, when, count, col

df1.select([count(when(isnan(c), c)).alias(c) for c in df1.columns]).show()

+-------------+-------+---------+----+------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+-----------+---------+-------------+-------------+---------+--------------+-----------+----------+-----------+-------+--------------+------------------+---------------------+------------------+-------------+--------------+-------------------+--------------+------------------+---------------------+------------------+-------------+--------------+-------------------+--------------+------------------+---------------------+------------------+-------------+--------------+-------------------+---------+---------+
|GLOBALEVENTID|SQLDATE|MonthYear|Year|FractionDate|Actor1Code|Actor1Name|Actor1CountryCode|Actor1KnownGrou

In [103]:
#### no need to run for re-running my code
df1.describe().show()

+-------+-------------------+------------------+-------------------+-------------------+----------+--------------------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+--------------------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+--------------------+------------------+--------------------+---------------------+------------------+------------------+-------------------+-------------------+------------------+--------------------+---------------------+------------------+------------------+--------------------+-------------------+------------------+--------------------+---------------------+------------------+----------

## Some SQL Anlaysis

In [89]:
#### no need to run for re-running my code
# EventCode is the raw CAMEO action code describing the action that Actor1 performed upon Actor2
spark.sql("""SELECT QuadClass, count(*) as count from df1 
            group by QuadClass order by count desc limit 10""").show()

+---------+--------+
|QuadClass|   count|
+---------+--------+
|        1|44668851|
|        4|10784200|
|        3| 9542781|
|        2| 8389866|
+---------+--------+



In [96]:
#### no need to run for re-running my code
spark.sql("""SELECT max(AvgTone) as AvgToneMax, min(AvgTone) as AvgToneMin from df1""").show()

+----------------+--------------------+
|             max|                 min|
+----------------+--------------------+
|9.99999999999999|-0.00010156355927...|
+----------------+--------------------+



In [19]:
#### no need to run for re-running my code
df1.write.parquet("s3://finalproject502/df.parquet")

In [20]:
sc.stop()
spark.stop()